In [ ]:
import os
from google.colab import drive

# Mount your Google Drive
drive.mount('/content/drive')

# Get the current working directory
current_dir = os.getcwd()

# Create a list of all the folders in the current working directory
folders = os.listdir(current_dir)

# Loop through the list of folders
for folder in folders:
  # Print the name of the folder
  print(folder)

Mounted at /content/drive
.config
drive
sample_data


In [ ]:
dataset_path = '/content/drive/MyDrive/dataset'

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader
import os
import torch
from torchvision import datasets, transforms
import torchvision
import torchvision.transforms as transforms
import torch.utils.data as data

In [ ]:
import torch

if torch.cuda.is_available():
  print("CUDA is available")
else:
  print("CUDA is not available")

print(torch.cuda.device_count())
device = None
for i in range(torch.cuda.device_count()):
  print(torch.cuda.get_device_name(i))
  device=torch.cuda.get_device_name(i)



CUDA is available
1
Tesla T4


In [ ]:
test_transforms = transforms.Compose([
    transforms.Resize((256,256)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])


train_transform = transforms.Compose([
    transforms.RandomRotation(20),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.RandomResizedCrop(224),
    transforms.Resize((256,256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

path = dataset_path
dataset = torchvision.datasets.ImageFolder(root=path)
print("len of dataset=",len(dataset) )
# Split the dataset into train and test
train_size = int(0.9 * len(dataset) )
test_size = len(dataset) - train_size
train_dataset, test_dataset = data.random_split(dataset, [train_size, test_size],generator=torch.Generator().manual_seed(42))

train_dataset.transform = train_transform
test_dataset.transform = test_transforms
# Define the dataloaders
print(len(train_dataset))
print(len(test_dataset))

train_loader = data.DataLoader(train_dataset, batch_size=32, shuffle=True)


test_loader = data.DataLoader(test_dataset, batch_size=32, shuffle=False)

trainloader=train_loader
testloader=test_loader

# Create a training and validation set
train_loader =train_loader
val_loader =test_loader


len of dataset= 922
829
93


In [ ]:
model_path = '/content/drive/MyDrive/model'
# Define a DenseNet model
model = torchvision.models.densenet161(pretrained=True)
# model.load_state_dict(torch.load(model_path+'/densenet161.pth'))

model.cuda()
# Train the model
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.001)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet161_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet161_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet161-8d451a50.pth" to /root/.cache/torch/hub/checkpoints/densenet161-8d451a50.pth
100%|██████████| 110M/110M [00:00<00:00, 268MB/s]


In [ ]:






for epoch in range(10):
  for i, (images, labels) in enumerate(train_loader):
    # Forward pass
    images = images.cuda()
    labels = labels.cuda()
    outputs = model(images)

    # Calculate loss
    loss = criterion(outputs, labels)

    # Backpropagate
    optimizer.zero_grad()
    loss.backward()

    # Update parameters
    optimizer.step()

    # Print loss
    if i % 100 == 0:
      print(loss.item())



0.000544100534170866
0.007484537083655596
0.0008593439706601202
0.00028540485072880983
0.00025132298469543457
0.0005401271628215909
0.0032382665667682886
0.00042626395588740706
0.00032849371200427413
0.002760413335636258


In [ ]:
# Evaluate the model
correct = 0
total = 0

for images, labels in val_loader:
    with torch.no_grad():
        images = images.cuda()
        labels = labels.cuda()
        outputs = model(images)
        
        predicted = outputs.argmax(dim=1)
        correct += (predicted == labels).sum().item()
        total += len(labels)

print('Accuracy: {}'.format(correct / total))

Accuracy: 0.967741935483871


In [ ]:
model_path = '/content/drive/MyDrive/model'

torch.save(model.state_dict(), model_path+'/densenet161.pth')